In [1]:
import numpy as np
import pandas as pd
import os

# File paths
technical_coeff_path = "C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/technical_coefficients/technical_coefficients_2019.csv"
price_volatility_path = "C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/price_data/II_PI_volatility.csv"
output_dir = "C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/results"

# Load data
technical_coefficients = pd.read_csv(technical_coeff_path, index_col=0)
price_volatility = pd.read_csv(price_volatility_path)
price_volatility.set_index('sector', inplace=True)  # New structure: 'sector' is the only index

# Placeholder for results
unweighted_impacts = []

# Iterate over each sector in price_volatility
for sector, row in price_volatility.iterrows():
    price_shock = row['price_volatility']

    # Skip zero volatility or excluded sectors
    if price_shock == 0:
        continue

    # Check if the row and column exist in the technical_coefficients DataFrame
    if sector not in technical_coefficients.index or sector not in technical_coefficients.columns:
        print(f"Skipping {sector} as it does not exist in technical_coefficients")
        continue

    # Extract A_EE (excluding the exogenous sector row and column)
    A_EE = technical_coefficients.drop(index=[sector]).drop(columns=[sector]).T

    # Extract A_XE (coefficients of the exogenous sector influencing endogenous sectors)
    A_XE = technical_coefficients[sector].drop(index=[sector]).values.reshape(-1, 1)

    # Identity matrix for endogenous sectors
    I = np.eye(A_EE.shape[0])

    # Calculate Leontief inverse
    try:
        leontief_inverse_endogenous = np.linalg.inv(I - A_EE.values)
    except np.linalg.LinAlgError:
        print(f"Singular matrix encountered for {sector}")
        continue

    # Calculate ΔP_E using the modified Leontief price model
    delta_P_E = leontief_inverse_endogenous @ (A_XE * price_shock)

    # Store unweighted impacts directly in a DataFrame
    unweighted_impact_series = pd.Series(delta_P_E.flatten(), index=A_EE.index, name=sector)
    unweighted_impacts.append(unweighted_impact_series)

# Convert results to DataFrame
unweighted_df = pd.DataFrame(unweighted_impacts)

# Ensure columns are in the same order as technical_coefficients columns (for standardization)
column_order = technical_coefficients.columns  # Capture the original order of columns
unweighted_df = unweighted_df.reindex(columns=column_order, fill_value=0)

# Save results to CSV
output_file = os.path.join(output_dir, 'unweighted_shock_impacts.csv')
unweighted_df.to_csv(output_file)

print(f"Unweighted impacts saved to {output_file}")


Unweighted impacts saved to C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/results\unweighted_shock_impacts.csv
